# Data Compression Using the Vitis Libraries

This final part of the lab introduces some of the higher-level functions of the Xilinx [data compression library](https://github.com/Xilinx/Vitis_Libraries/tree/b658aa5cd262d080048526ce931d4570cb931a36/data_compression) which is part of the set of [Vitis Libraries](https://github.com/Xilinx/Vitis_Libraries). We're going to use a pre-compiled bitstream from the [PYNQ Alveo Examples](https://github.com/Xilinx/Alveo-PYNQ) which is designed to show some of the features of the library. In particular it has hardware kernels for:

 * LZ4 compression
 * DEFLATE compression

For this lab we're going to focus on the LZ4 compression algorithm as it requires only a single hardware accelerator rather than the three required for DEFLATE. If you want to learn more about DEFLATE the instructions in the [resources notebook](../Resources.ipynb) will guide you through installing the complete set of PYNQ Alveo Examples

LZ4 is a block-based compression library optimized for speed and parallel access. The first thing we need to do, as with the previous labs, is load the bitstream:

In [ ]:
import pynq

ol = pynq.Overlay('compression.xclbin')

Which will allow us to inspect its contents:

In [ ]:
ol?

As you can see there are multiple types of kernel with each type having two instances. For this notebook we are only interested in the `xilLz4Compress_2` kernel which we will use to introduce the concepts behind the Vitis data compression library.

In [ ]:
compress = ol.xilLz4Compress_2
compress.signature

Before constructing the arguments to pass to a hardware kernel we need to understand how the compression library is structured. The compress kernel
is composed of 8 separate internal pipelines with a splitter and combiner at each end:

![compression pipeline](img/lzx_comp.png)

With this logic we can start thinking of the formats for each argument:

 * `in_r` - the input data array consisting of 8 buffers of the block size arranged contiguously
 * `out_r` - the output data array consisting of 8 buffers of the block size arranged contiguously
 * `compressed_size` - an array of size 8 that will contain the sizes of the blocks after compression
 * `in_block_size` - an array of size 8 that contains the uncompressed sizes of the blocks (as the data may not fill the whole block)
 * `block_size_in_kb` - the block size
 * `input_size` - the total size of the input
 
The LZ4 format has 4 possible block sizes from 64 KB to 4 MB. For this example we'll set the block size as 1 MB. Using this as a size we can create all of the buffers. Note that for this accelerator all of the arrays should be in `bank1` as we can see from the `mem` attribute of each entry in the `args` dictionary.

In [ ]:
compress.args

Note that unlike the previous labs this design uses multiple memory banks - one for each compressor. The hardware block we've chosen is using `bank1` so we need to make sure that any buffers we allocate are in that range. The `target=` parameter is used to select a bank when allocating buffers. `target` can be used for a number of things but in this case we want to specify a memory bank. We can get a bank object from the Overlay - `bank1` in this case.

Each argument is an array of 8 buffers or values - one for each of the pipelines in the accelerator

In [ ]:
BLOCK_SIZE = 1024 * 1024

in_buffers = pynq.allocate((8, BLOCK_SIZE), 'u1', target=ol.bank1)
out_buffers = pynq.allocate((8, BLOCK_SIZE), 'u1', target=ol.bank1)
compressed_size = pynq.allocate((8,), 'u4', target=ol.bank1)
uncompressed_size = pynq.allocate((8,), 'u4', target=ol.bank1)

We've included a reasonably sized file, `test_data.bin`, which is at least somewhat compressible.

In [ ]:
with open('test_data.bin', 'rb') as f:
    test_data = f.read()

To get the data into our input buffers we need to format the data. We'll use NumPy to do this. First we `reshape` the array to make it a single linear object then we assign a `memoryview` of the data to the reshaped array. The `memoryview` is needed here because NumPy treats a string as a text object to be parsed, not as raw data to copy.

Despite all the function calls this results in a single copy of the data from the `test_data` string to our PYNQ buffer. This demonstrates one of the powers of numpy in offering zero-copy ways of slicing and recasting data from python.

In [ ]:
in_buffers.reshape(8*BLOCK_SIZE)[:] = memoryview(test_data)[0:8*BLOCK_SIZE]

The uncompressed size of each block is just 1 MB

In [ ]:
uncompressed_size[:] = BLOCK_SIZE

And as a final step we need to sync the input buffers with the device

In [ ]:
uncompressed_size.sync_to_device()
in_buffers.sync_to_device()

Everything is now set up to call the accelerator

In [ ]:
compress.call(in_buffers, out_buffers,
             compressed_size, uncompressed_size,
              1024, 8 * BLOCK_SIZE)

To get the data back we first need to sync the buffer containing the sizes of the blocks

In [ ]:
compressed_size.sync_from_device()
compressed_size

And we can now sync the output buffers - note that we only need to sync the part of the buffer that we know is filled

In [ ]:
for i in range(8):
    out_buffers[i,0:compressed_size[i]].sync_from_device()

To verify we have the correct result we can to pass the compressed data back an decompress it in software. The `lz4` package provides a block-level API we can use for decompressing our results.

In [ ]:
import lz4.block

decompressed_data = b''
for i in range(8):
    decompressed_data += lz4.block.decompress(out_buffers[i, 0:compressed_size[i]],
                                              uncompressed_size=1024*1024)
    
decompressed_data == test_data[0:8*1024*1024]

To get an idea of the compression ratio we can concatenate the length of the compressed blocks and device by the 8 MB we started with

In [ ]:
sum(compressed_size) / sum(uncompressed_size)

## Exercises

This notebook has introduced the very basics of the compression library and shown how to compress a single set of blocks. With that in mind:

 1. Compress a larger file by overlapping compute and communication
 2. Use the second LZ4 compression accelerator to compress two files simultaneously - note that the PYNQ API is thread-safe if you want to go down that route
 3. (Optional) Create a compressor for a complete file in the [LZ4 Frame Format](https://github.com/lz4/lz4/blob/master/doc/lz4_Frame_format.md). We've included a magic number for a header without any checksums and 1 MB block size to get you started. The lz4.frame library can be used to check your results
 
Again, once you've finished you'll want to shut down this notebook.

In [ ]:
LZ4_MAGIC = b'\x04\x22\x4D\x18'
LZ4_HEADER = LZ4_MAGIC + b'\x60\x60\x51'

Copyright (C) 2020 Xilinx, Inc